In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages

from tqdm import tqdm


import sys
import spikeinterface as si
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import probeinterface

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probegroup
from probeinterface import generate_dummy_probe, generate_linear_probe
from probeinterface import write_probeinterface, read_probeinterface
from probeinterface import write_prb, read_prb
from torch.nn.functional import max_pool1d


import torch.nn.functional as F
from pathlib import Path

import time

In [17]:
def get_spike_inf_phy(file_path):
    cluster_inf = pd.read_csv(f'{file_path}/phy_folder_for_kilosort/cluster_info.tsv', sep = '\t')
    cluster_inf.columns = ['cluster_id', 'amp',	'amplitude_cutoff',	'amplitude_cv_median', 'amplitude_cv_range',	
                        'amplitude_median', 'ch', 'channel_group', 'depth', 'drift_mad', 
                        'drift_ptp',	'drift_std', 'firing_range', 'fr', 'group',	'isi_violations_count',	'isi_violations_ratio',	
                        'n_spikes', 'presence_ratio', 'rp_contamination', 'rp_violations', 'sd_ratio', 'sh', 'si_unit_id', 'sliding_rp_violation', 
                        'snr', 'sync_spike_2', 'sync_spike_4', 'sync_spike_8']

    cluster_inf = cluster_inf[cluster_inf['group'] == 'good']

    cluster_inf['cluster_id'] = cluster_inf['cluster_id'].astype(str)
    cluster_inf['position_1'] = None
    cluster_inf['position_2'] = None

    spike_clusters = pd.DataFrame(np.load(file_path + "/phy_folder_for_kilosort/spike_clusters.npy").astype(str))
    spike_positions = pd.DataFrame(np.load(file_path + "/kilosort4/sorter_output/spike_positions.npy").astype(float))
    spike_templates = pd.DataFrame(np.load(file_path + "/phy_folder_for_kilosort/spike_templates.npy"))
    spike_times = pd.DataFrame(np.load(file_path + "/phy_folder_for_kilosort/spike_times.npy").astype(int))

    spike_inf = pd.concat((spike_clusters, spike_positions, spike_templates, spike_times), axis=1)
    spike_inf.columns = ['cluster', 'position_1', 'position_2', 'templates', 'time']
    spike_inf = spike_inf[spike_inf['cluster'].isin(cluster_inf['cluster_id'].unique())]

    for i in spike_inf['cluster'].value_counts().index:
        temp = spike_inf[spike_inf['cluster'] == i]
        cluster_inf.loc[cluster_inf['cluster_id'] == i, 'position_1'] = np.mean(temp['position_1'])
        cluster_inf.loc[cluster_inf['cluster_id'] == i, 'position_2'] = np.mean(temp['position_2'])
    return spike_inf, cluster_inf

In [18]:
spike_inf, cluster_inf = get_spike_inf_phy(file_path='/media/ubuntu/sda/Spike_Sorting/paper_architecture/01_real_data/03_Neuropixel_384_channels_visual_stimuli/spike_sorting/810755809')

FileNotFoundError: [Errno 2] No such file or directory: '/media/ubuntu/sda/Spike_Sorting/paper_architecture/01_real_data/03_Neuropixel_384_channels_visual_stimuli/spike_sorting/810755809/phy_folder_for_kilosort/cluster_info.tsv'

In [ ]:
spike_inf.to_csv("/media/ubuntu/sda/Spike_Sorting/paper_architecture/01_real_data/03_Neuropixel_384_channels_visual_stimuli/spike_sorting/810755809/spike_inf.tsv", sep = '\t')

In [ ]:
cluster_inf.to_csv("/media/ubuntu/sda/Spike_Sorting/paper_architecture/01_real_data/03_Neuropixel_384_channels_visual_stimuli/spike_sorting/810755809/cluster_inf.csv")